# Historical CalFire - Camera Matching

This notebook takes data from the CalFire dataset (https://www.fire.ca.gov/incidents/) under *All Incident Data*(https://www.fire.ca.gov/imapdata/mapdataall.csv) at the bottom of the page. It deduces which cameras may have seen that those fires using the camera metadata.  This creates two columns which represent which cameras would have seen each fire in the San Diego region if cameras are assumed to be able to see 25mi out and 15mi out.  This data is exported to the file "fire_camera_matches.csv"

The CalFire historical fire data should be placed in the data folder

In [87]:
import pandas as pd
import geopandas
from shapely.geometry import Polygon, LineString, Point
from haversine import haversine, Unit
import plotly.express as px
import numpy as np
from dateutil.parser import parse
import datetime as dt

In [88]:
cam_meta = pd.read_csv("../../data/processed/camera_metadata_hpwren.csv")
cam_meta = cam_meta[['camera_abbrev', 'direction', 'lat', 'long', 'center_angle']]
cam_meta = cam_meta.loc[(cam_meta['direction'] == 'north') | (cam_meta['direction'] == 'east') | (cam_meta['direction'] == 'south') |(cam_meta['direction'] == 'west')].reset_index(drop=True)
cam_meta['center_angle'] = cam_meta['center_angle'].fillna(0)

In [89]:
def find_fov_right(direction, lat, long, fill_dist, c_angle):
    f_dist = fill_dist / np.sin(np.pi/4)
    fdisty = f_dist * np.sin(c_angle+(np.pi/4))
    fdistx = f_dist * np.cos(c_angle+(np.pi/4))
    if direction == "north":
        return (long+fdistx, lat+fdisty)
    if direction == "east":
        return (long+fdistx, lat-fdisty)
    if direction == "south":
        return (long-fdistx, lat-fdisty)
    if direction == "west":
        return (long-fdistx, lat+fdisty)
    
def find_fov_left(direction, lat, long, fill_dist, c_angle):
    f_dist = fill_dist / np.sin(np.pi/4)
    fdisty = f_dist * np.sin(c_angle+(np.pi/4))
    fdistx = f_dist * np.cos(c_angle+(np.pi/4))
    if direction == "north":
        return (long-fdistx, lat+fdisty)
    if direction == "east":
        return (long+fdistx, lat+fdisty)
    if direction == "south":
        return (long+fdistx, lat-fdisty)
    if direction == "west":
        return (long-fdistx, lat-fdisty)


In [90]:
df_fires = pd.read_csv("data/mapdataall.csv")
in_date = parse(df_fires['incident_date_last_update'][0])
df_fires['date'] = df_fires.apply(lambda x : (parse(x['incident_date_created'])).strftime('%Y%m%d'), axis = 1)
df_fires_sd = df_fires.loc[df_fires['incident_county'].isin(['San Diego', 'Mexico, San Diego', 'Riverside, San Diego', 'Orange'])].reset_index(drop=True)
df_fires_sd = df_fires_sd.loc[(df_fires_sd['incident_longitude'] < -115) & (df_fires_sd['incident_latitude'] < 34.5)]
df_fires_sd = df_fires_sd.reset_index(drop=True)

In [91]:
df_fires_sd.head()

,incident_name,incident_is_final,incident_date_last_update,incident_date_created,incident_administrative_unit,incident_administrative_unit_url,incident_county,incident_location,incident_acres_burned,incident_containment,...,incident_type,incident_id,incident_url,incident_date_extinguished,incident_dateonly_extinguished,incident_dateonly_created,is_active,calfire_incident,notification_desired,date
0,Border Fire,Y,2013-05-23T08:30:00Z,2013-05-22T16:14:00Z,CAL FIRE San Diego Unit,NaN,San Diego,Off Alta Rd in the Otay Mesa area,100.0,100.0,...,NaN,e077e091-f890-477c-b948-eb23774b432e,https://osfm.fire.ca.gov/incidents/2013/5/22/b...,2013-05-23T08:30:00Z,2013-05-23,2013-05-22,N,True,False,20130522
1,San Felipe Fire,Y,2013-05-26T17:45:00Z,2013-05-23T12:20:00Z,CAL FIRE San Diego Unit,NaN,San Diego,"San Felipe Road, north of Highway 78, East of ...",2781.0,100.0,...,NaN,859479e3-918c-42c6-bb74-5fdc17930a16,https://osfm.fire.ca.gov/incidents/2013/5/23/s...,2013-05-26T17:45:00Z,2013-05-26,2013-05-23,N,True,False,20130523
2,General Fire,Y,2013-05-31T06:15:00Z,2013-05-26T12:04:00Z,CAL FIRE San Diego Unit,NaN,San Diego,"Banner Grade area, south of Hwy 78 & east of H...",1271.0,100.0,...,NaN,07a5397c-a665-4f84-9a82-5f689cb2c8f3,https://osfm.fire.ca.gov/incidents/2013/5/26/g...,2013-05-31T06:15:00Z,2013-05-31,2013-05-26,N,True,False,20130526
3,Chariot Fire,Y,2013-07-15T06:15:00Z,2013-07-06T12:55:00Z,CAL FIRE San Diego Unit / Cleveland National F...,NaN,San Diego,"off Sunrise Hwy, 9 miles southeast of Julian",7055.0,100.0,...,NaN,ee19b2ec-a96a-4738-994e-fb3ea016e053,https://osfm.fire.ca.gov/incidents/2013/7/6/ch...,2013-07-15T06:15:00Z,2013-07-15,2013-07-06,N,True,False,20130706
4,Flume Fire,Y,2013-07-28T17:50:00Z,2013-07-28T09:43:00Z,CAL FIRE San Diego Unit,NaN,San Diego,Barrett Lake Road and Highway 94,25.0,100.0,...,NaN,c6cfeed9-55bb-4fdb-8a52-e4d947dba721,https://osfm.fire.ca.gov/incidents/2013/7/28/f...,2013-07-28T17:50:00Z,2013-07-28,2013-07-28,N,True,False,20130728


In [92]:
fill_dist = .2777 #25miles out assumption
cam_meta['fov_right'] = cam_meta.apply(lambda x: find_fov_right(x['direction'], x['lat'], x['long'], fill_dist, x['center_angle']), axis=1)
cam_meta['fov_left'] = cam_meta.apply(lambda x: find_fov_left(x['direction'], x['lat'], x['long'], fill_dist, x['center_angle']), axis=1)

left_edges = {'north': 'west', 'east': 'north', 'south':'east', 'west': 'south'}
right_edges = {'north': 'east', 'east':'south', 'south':'west', 'west':'north'}
intersections = []
counts = []

for i in range(len(df_fires_sd)):
    direction = cam_meta['direction'][i]
    intersection = []
    long = df_fires_sd['incident_longitude'][i]
    lat = df_fires_sd['incident_latitude'][i]
    for j in range(len(cam_meta)):
        c2 = (cam_meta['long'][j], cam_meta['lat'][j])
        r2 = cam_meta['fov_right'][j]
        l2 = cam_meta['fov_left'][j]
        if geopandas.GeoSeries(Point(long, lat)).intersects(geopandas.GeoSeries(Polygon([c2, r2, l2])))[0] == True:
            if cam_meta['camera_abbrev'][j] != cam_meta['camera_abbrev'][i]:
                intersection.append((cam_meta['camera_abbrev'][j], cam_meta['direction'][j]))
    if len(intersection) > 0:
        intersections.append(intersection)
        counts.append(len(intersection))
    else:
        intersections.append(0)
        counts.append(0)    
        
df_fires_sd['intersections_25mi'] = intersections
df_fires_sd['num_intersections_25mi'] = counts



fill_dist = .1666 #15miles out assumption
cam_meta['fov_right'] = cam_meta.apply(lambda x: find_fov_right(x['direction'], x['lat'], x['long'], fill_dist, x['center_angle']), axis=1)
cam_meta['fov_left'] = cam_meta.apply(lambda x: find_fov_left(x['direction'], x['lat'], x['long'], fill_dist, x['center_angle']), axis=1)
intersections = []
counts = []

for i in range(len(df_fires_sd)):
    direction = cam_meta['direction'][i]
    intersection = []
    long = df_fires_sd['incident_longitude'][i]
    lat = df_fires_sd['incident_latitude'][i]
    for j in range(len(cam_meta)):
        c2 = (cam_meta['long'][j], cam_meta['lat'][j])
        r2 = cam_meta['fov_right'][j]
        l2 = cam_meta['fov_left'][j]
        if geopandas.GeoSeries(Point(long, lat)).intersects(geopandas.GeoSeries(Polygon([c2, r2, l2])))[0] == True:
            if cam_meta['camera_abbrev'][j] != cam_meta['camera_abbrev'][i]:
                intersection.append((cam_meta['camera_abbrev'][j], cam_meta['direction'][j]))
    if len(intersection) > 0:
        intersections.append(intersection)
        counts.append(len(intersection))
    else:
        intersections.append(0)
        counts.append(0)    
        
df_fires_sd['intersections_15mi'] = intersections
df_fires_sd['num_intersections_15mi'] = counts

In [93]:
df_out= df_fires_sd[['incident_name', 'incident_date_created', 'incident_county', 'incident_location', 'incident_longitude', 'incident_latitude', 'intersections_25mi', 'num_intersections_25mi', 'intersections_15mi', 'num_intersections_15mi']]

In [94]:
df_out.to_csv('data/fire_camera_matches.csv', index=False)

## Extra Visualizations

In [95]:
df = df_out['num_intersections_25mi']
fig = px.histogram(df, x="num_intersections_25mi",
                   title='Hpwren Cameras Coverge at 25mi',
                   labels={'num_intersections_25mi':'# of Cameras that Would See Fire'}, text_auto=True)
fig.update_traces(textposition='outside', selector=dict(type='histogram'), textfont={'size': 14})
fig.show()

In [96]:
df = df_out['num_intersections_15mi']
fig = px.histogram(df, x="num_intersections_15mi",
                   title='Hpwren Cameras Coverge at 15mi',
                   labels={'num_intersections_15mi':'# of Cameras that Would See Fire'},
                   text_auto=True)
fig.update_traces(textposition='outside', selector=dict(type='histogram'), textfont={'size': 14})
fig.show()